In [29]:

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1, verbose=True)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "Top Gun",
        "answer": """
            🛩️👨‍✈️🔥
        """,
    },
    {
        "question": "Godfather",
        "answer": """
            👨‍👨‍👦🔫🍝
        """,
    },
    {
        "question": "Jurassic Park",
        "answer": """
            🦖🌴🦕
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. "),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({
    "question": question
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

invoke_chain("star wars")


content='\n            🌌⚔️🤖\n        '


In [30]:
invoke_chain("Dune")

content='🌌🏜️🐛'


In [31]:
invoke_chain("What was the previous movie I asked?")

content='The previous movie you asked about was "Star Wars."'
